In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input,Lambda,Dense,Flatten
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.models import Sequential
from keras.models import load_model
from glob import glob
from keras.models import Model

In [4]:
image_size=[224,224]

train_path='chest_xray/train'
valid_path='chest_xray/test'

In [5]:
vgg=VGG16(input_shape= image_size + [3], weights='imagenet',include_top=False)

In [6]:
# in Transfer learning we dont neede to train existing weights
for layer in vgg.layers:
    layer.trainable=False
    

In [7]:
# find out the number of classes
folders=glob('chest_xray/train/*')

In [8]:
folders

['chest_xray/train\\NORMAL', 'chest_xray/train\\PNEUMONIA']

In [9]:
x=Flatten()(vgg.output)

In [10]:
x

<tf.Tensor 'flatten_1/Reshape:0' shape=(None, None) dtype=float32>

In [11]:
prediction=Dense(len(folders),activation='softmax')(x)

In [12]:
prediction

<tf.Tensor 'dense_1/Softmax:0' shape=(None, 2) dtype=float32>

In [13]:
# create a model
model=Model(inputs=vgg.input,outputs=prediction)

In [14]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [15]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)


In [17]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [18]:
training_set=train_datagen.flow_from_directory('chest_xray/train',
                                              target_size=(224,224),
                                              batch_size=32,
                                              class_mode='categorical')


Found 5232 images belonging to 2 classes.


In [19]:
test_set=train_datagen.flow_from_directory('chest_xray/test',
                                              target_size=(224,224),
                                              batch_size=32,
                                              class_mode='categorical')

Found 624 images belonging to 2 classes.


In [ ]:
# fit the model
r=model.fit_generator(
                       training_set,
                        validation_data=test_set,
                         epochs=5,
                        steps_per_epoch=len(training_set),
                        validation_steps=len(test_set)
                        )

Epoch 1/5
164/164 [==============================] - 3401s 21s/step - loss: 0.1965 - accuracy: 0.9260 - val_loss: 0.5627 - val_accuracy: 0.8109
Epoch 2/5
 32/164 [====>.........................] - ETA: 34:00 - loss: 0.1194 - accuracy: 0.9590

In [ ]:
# loss
plt.plot(r.history['loss'],label='train loss')
plt.plot(r.history['val_loss'],label='val loss')
plt.legend()
plt.show()
plt.savefig('lossValue Loss')

In [ ]:
# accuracy
plt.plot(r.history['accuracy'],label='train accuracy')
plt.plot(r.history['val_accuracy'],label='val acc')
plt.legend()
plt.show()
plt.savefig('Accuracy')